In [9]:
import os
from utils import write_tools
import zarr


array_cube_side = 2048
desired_cube_side = 512
chunk_size = 64
raw_ncar_folder_path = '/home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt'
use_dask = True
dest_folder_name = "sabl2048b" # B is the high-rate data
write_type = "prod" # or "back" for backup

n_dask_workers = 4 # For Dask rechunking

# Kernel dies with Sciserver large jobs resources as of Aug 2023. Out of memory IMO
num_threads = 8  # For writing to FileDB
dask_local_dir = '/home/idies/workspace/turb/data02_02'

timestep_nr = 3

In [10]:
folders=write_tools.list_fileDB_folders()

# Avoiding 7-2 and 9-2 - they're too full as of May 2023
folders.remove("/home/idies/workspace/turb/data09_02/zarr/")
folders.remove("/home/idies/workspace/turb/data07_02/zarr/")

for i in range(len(folders)):
    folders[i] += dest_folder_name + "_" + str(i + 1).zfill(2) + "_" + write_type + "/"

In [11]:
range_list = [] # Where chunks start and end. Needed for Mike's code to find correct chunks to access
smaller_size = 512
outer_dim = []

for i in range(4):
    mid_dim = []
    for j in range(4):
        inner_dim = []

        for k in range(4):
            a = []
            a.append([i * smaller_size, (i + 1) * smaller_size])
            a.append([j * smaller_size, (j + 1) * smaller_size])
            a.append([k * smaller_size, (k + 1) * smaller_size])

            range_list.append(a)

        mid_dim.append(inner_dim)

    outer_dim.append(mid_dim)

In [12]:
chunk_morton_mapping = write_tools.get_chunk_morton_mapping(range_list, dest_folder_name)
flattened_node_assgn = write_tools.flatten_3d_list(write_tools.node_assignment(4))

In [13]:
dests = []

for i in range(len(range_list)):
#     for j in range(4):
#         for k in range(4):
    min_coord = [a[0] for a in range_list[i]]
    max_coord = [a[1] - 1 for a in range_list[i]]

    morton = (write_tools.morton_pack(array_cube_side, min_coord[2], min_coord[1], min_coord[0]), write_tools.morton_pack(array_cube_side, max_coord[2], max_coord[1], max_coord[0]))

    chunk_name = write_tools.search_dict_by_value(chunk_morton_mapping, morton)

    idx = int(chunk_name[-2:].lstrip('0'))

    filedb_index = flattened_node_assgn[idx - 1] - 1

    destination = os.path.join(folders[filedb_index], dest_folder_name + str(idx).zfill(2) + "_" + str(timestep_nr).zfill(3) + ".zarr")

    dests.append(destination)

In [14]:
dests

['/home/idies/workspace/turb/data01_01/zarr/sabl2048b_01_prod/sabl2048b01_003.zarr',
 '/home/idies/workspace/turb/data02_01/zarr/sabl2048b_02_prod/sabl2048b02_003.zarr',
 '/home/idies/workspace/turb/data09_01/zarr/sabl2048b_09_prod/sabl2048b09_003.zarr',
 '/home/idies/workspace/turb/data10_01/zarr/sabl2048b_10_prod/sabl2048b10_003.zarr',
 '/home/idies/workspace/turb/data03_01/zarr/sabl2048b_03_prod/sabl2048b03_003.zarr',
 '/home/idies/workspace/turb/data04_01/zarr/sabl2048b_04_prod/sabl2048b04_003.zarr',
 '/home/idies/workspace/turb/data11_01/zarr/sabl2048b_11_prod/sabl2048b11_003.zarr',
 '/home/idies/workspace/turb/data12_01/zarr/sabl2048b_12_prod/sabl2048b12_003.zarr',
 '/home/idies/workspace/turb/data05_02/zarr/sabl2048b_17_prod/sabl2048b17_003.zarr',
 '/home/idies/workspace/turb/data06_02/zarr/sabl2048b_18_prod/sabl2048b18_003.zarr',
 '/home/idies/workspace/turb/data03_03/zarr/sabl2048b_25_prod/sabl2048b25_003.zarr',
 '/home/idies/workspace/turb/data04_03/zarr/sabl2048b_26_prod/sab

In [27]:
import xarray as xr

data_xr = xr.open_dataset('~/Downloads/jhd.000.nc')
data_xr

<xarray.Dataset>
Dimensions:  (nnz: 2048, nny: 2048, nnx: 2048)
Dimensions without coordinates: nnz, nny, nnx
Data variables:
    u        (nnz, nny, nnx) float32 ...
    v        (nnz, nny, nnx) float32 ...
    w        (nnz, nny, nnx) float32 ...
    t        (nnz, nny, nnx) float32 ...
    p        (nnz, nny, nnx) float32 ...
    e        (nnz, nny, nnx) float32 ...
Attributes:
    Simulation conducted by:  Peter Sullivan, pps@ucar.edu
    Dataset built by:         Edward Patton, patton@ucar.edu
    Affiliation:              NCAR/MMM
    Code:                     NCAR-LES
    Case Description:         GABLS 1, Cooling Rate: 0.25 K/hr
    Created:                  2023-07-08 07:56:23 -0600 UTC

In [28]:
t = zarr.open_group(dests[0], mode='a')
t.info

KeyError: '.zgroup'

In [ ]:
curr_arr = t[chosen_var][...]
curr_arr.shape

In [ ]:
reshaped_data = curr_arr.reshape(curr_arr.shape + (1,))
reshaped_data.shape

In [ ]:
del t[chosen_var]

In [ ]:
t.create_dataset(chosen_var, data=reshaped_data, chunks=(64, 64, 64, 1), compressor=None)